# Regression with Keras and CNNs
https://www.pyimagesearch.com/2019/01/28/keras-regression-and-cnns/

In [1]:
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from pyimagesearch import datasets
from keras.layers import Input
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
import numpy as np
import locale
import os
import glob
import cv2

Using TensorFlow backend.


## Settings

In [2]:
dataset = '../../datasets/Houses-dataset/Houses Dataset/'

## Function definition

In [3]:
def load_house_images(df, inputPath):
    # Initialize our images array (i.e., the house images themselves)
    images = []
    
    # Loop over the indexes of the houses
    for i in df.index.values:
        # Find the four images for the house and sort the file paths,
        # ensuring the four are always in the *same order*
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        housePaths = sorted(list(glob.glob(basePath)))
        
        # Initialize our list of input images along with the output image
        # after *combining* the four input images
        inputImages = []
        outputImage = np.zeros((64, 64, 3), dtype="uint8")
        
        # Loop over the input house paths
        for housePath in housePaths:
            # Load the input image, resize it to be 32 32, and then update the list of input images
            image = cv2.imread(housePath)
            image = cv2.resize(image, (32, 32))
            inputImages.append(image)
            
        # Tile the four input images in the output image such the first
        # image goes in the top-right corner, the second image in the
        # top-left corner, the third image in the bottom-right corner,
        # and the final image in the bottom-left corner
        outputImage[0:32, 0:32] = inputImages[0]
        outputImage[0:32, 32:64] = inputImages[1]
        outputImage[32:64, 32:64] = inputImages[2]
        outputImage[32:64, 0:32] = inputImages[3]
        
        # Add the tiled image to our set of images the network will be trained on
        images.append(outputImage)
        
    # Return our set of images
    return np.array(images)

In [4]:
def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
    # Initialize the input shape and channel dimension, assuming TensorFlow/channels-last ordering
    inputShape = (height, width, depth)
    chanDim = -1
    
    # Define the model input
    inputs = Input(shape=inputShape)

    # Loop over the number of filters
    for (i, f) in enumerate(filters):
        # If this is the first CONV layer then set the input appropriately
        if i == 0:
            x = inputs
        # CONV => RELU => BN => POOL
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        
    # Flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)
    
    # Apply another FC layer, this one to match the number of nodes coming out of the
    # Multilayer Perceptron (MLP)
    x = Dense(4)(x)
    x = Activation("relu")(x)
    
    # Check to see if the regression node should be added
    if regress:
        x = Dense(1, activation="linear")(x)
    
    # Construct the CNN
    model = Model(inputs, x)
    
    # Return the CNN
    return model

## Training

In [5]:
# Construct the path to the input .txt file that contains information
# on each house in the dataset and then load the dataset
inputPath = os.path.sep.join([dataset, "HousesInfo.txt"])
df = datasets.load_house_attributes(inputPath)

# Load the house images and then scale the pixel intensities to the range [0, 1]
images = load_house_images(df, dataset)
images = images / 255.0

# Partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

In [6]:
# Find the largest house price in the training set and use it to scale our
# house prices to the range [0, 1] (will lead to better training and convergence)
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice

# Create our Convolutional Neural Network and then compile the model
# using mean absolute percentage error as our loss, implying that we
# seek to minimize the absolute percentage difference between our
# price *predictions* and the *actual prices*
model = create_cnn(64, 64, 3, regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

# Train the model
print("Training model...")
model.fit(trainImagesX, trainY, validation_data=(testImagesX, testY), epochs=200, batch_size=8)

Training model...
Train on 271 samples, validate on 91 samples
Epoch 1/200
271/271 [==============================] - 1s 4ms/step - loss: 2587.0449 - val_loss: 2873.6024
Epoch 2/200
271/271 [==============================] - 0s 2ms/step - loss: 1672.9706 - val_loss: 4966.3087
Epoch 3/200
271/271 [==============================] - 0s 2ms/step - loss: 1359.3165 - val_loss: 6235.6242
Epoch 4/200
271/271 [==============================] - 0s 2ms/step - loss: 798.3174 - val_loss: 11256.7455
Epoch 5/200
271/271 [==============================] - 0s 2ms/step - loss: 730.9826 - val_loss: 10538.1426
Epoch 6/200
271/271 [==============================] - 0s 2ms/step - loss: 924.7960 - val_loss: 10551.9717
Epoch 7/200
271/271 [==============================] - 0s 2ms/step - loss: 683.4718 - val_loss: 10488.4716
Epoch 8/200
271/271 [==============================] - 0s 2ms/step - loss: 487.7078 - val_loss: 9035.7322
Epoch 9/200
271/271 [==============================] - 0s 2ms/step - loss: 491.343

271/271 [==============================] - 1s 2ms/step - loss: 64.5685 - val_loss: 70.8264
Epoch 79/200
271/271 [==============================] - 0s 2ms/step - loss: 67.6799 - val_loss: 70.3139
Epoch 80/200
271/271 [==============================] - 0s 2ms/step - loss: 65.1915 - val_loss: 71.1602
Epoch 81/200
271/271 [==============================] - 0s 2ms/step - loss: 65.2955 - val_loss: 70.4853
Epoch 82/200
271/271 [==============================] - 0s 2ms/step - loss: 62.3240 - val_loss: 71.3735
Epoch 83/200
271/271 [==============================] - 1s 2ms/step - loss: 63.9339 - val_loss: 71.1328
Epoch 84/200
271/271 [==============================] - 0s 2ms/step - loss: 64.8049 - val_loss: 71.4405
Epoch 85/200
271/271 [==============================] - 0s 2ms/step - loss: 69.4813 - val_loss: 71.4976
Epoch 86/200
271/271 [==============================] - 0s 2ms/step - loss: 64.5309 - val_loss: 70.5585
Epoch 87/200
271/271 [==============================] - 0s 2ms/step - loss: 6

271/271 [==============================] - 0s 2ms/step - loss: 44.0497 - val_loss: 55.6289
Epoch 157/200
271/271 [==============================] - 1s 2ms/step - loss: 38.4971 - val_loss: 57.1318
Epoch 158/200
271/271 [==============================] - 1s 2ms/step - loss: 39.4241 - val_loss: 56.5587
Epoch 159/200
271/271 [==============================] - 1s 2ms/step - loss: 40.6807 - val_loss: 58.8067
Epoch 160/200
271/271 [==============================] - 1s 2ms/step - loss: 41.0853 - val_loss: 55.5010
Epoch 161/200
271/271 [==============================] - 0s 2ms/step - loss: 39.3300 - val_loss: 52.3961
Epoch 162/200
271/271 [==============================] - 1s 2ms/step - loss: 40.9031 - val_loss: 52.6625
Epoch 163/200
271/271 [==============================] - 0s 2ms/step - loss: 38.7639 - val_loss: 51.7585
Epoch 164/200
271/271 [==============================] - 1s 2ms/step - loss: 39.6237 - val_loss: 51.3575
Epoch 165/200
271/271 [==============================] - 1s 2ms/step 

## Evaluation

In [7]:
# Make predictions on the testing data
preds = model.predict(testImagesX)

# Compute the difference between the *predicted* house prices and the *actual* house prices,
# then compute the percentage difference and the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

# Compute the mean and standard deviation of the absolute percentage difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

# Finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("Avg. house price: {}, std house price: {}".format(locale.currency(df["price"].mean(), grouping=True),
                                                         locale.currency(df["price"].std(), grouping=True)))
print("Mean: {:.2f}%, std: {:.2f}%".format(mean, std))

Avg. house price: $533,388.27, std house price: $493,403.08
Mean: 57.72%, std: 54.66%
